In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os
import math
import time
import signal
from tqdm import tqdm
from contextlib import contextmanager

In [4]:
@contextmanager
def timeout(duration):
    def timeout_handler(signum, frame):
        raise Exception(f'block timedout after {duration} seconds')
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(duration)
    yield
    signal.alarm(0)

In [50]:
def scroll_down(driver):
    """A method for scrolling the page."""
    
    height = "offsetHeight"

    # Get scroll height.
    last_height = driver.execute_script("return document.body."+height)

    while True:
        
        
        print("scrolling...")
        # Scroll down to the bottom.
        driver.execute_script("window.scrollTo(0, document.body."+height+");")

        # Wait to load the page.
        print("waiting")
        time.sleep(3)
        
        try:
            # Calculate new scroll height and compare with last scroll height.
            with timeout(20):
                new_height = driver.execute_script("return document.body."+height)
        except Exception:
            print('stopped scrolling - time')
            break
            

        if new_height == last_height:
            
            print('stopped scrolling - reached end')
            break

        last_height = new_height
    
    return driver

In [51]:
def get_all_author_profile_hyperlink(hyperlink):

    driver = webdriver.Chrome()

    driver.get(hyperlink)

    # scroll to laod entire page
    driver = scroll_down(driver)

    # get author hyperlinks
    author_profile_hyperlinks = []
    elements = driver.find_elements_by_class_name("poetNameDatePlace")

    for element in tqdm(elements):
        try:
            authoer_profile = element.find_elements_by_tag_name('a')[0].get_attribute('href')
            if authoer_profile!=None or (type(authoer_profile)==str and authoer_profile!=''):
                author_profile_hyperlinks.append(authoer_profile)
            else:
                pass
        except IndexError:
            pass
    driver.close()
    
    return author_profile_hyperlinks

In [52]:
def get_ghazal_hyperlinks_from_author_page(author_page_hyperlink):
    driver = webdriver.Chrome()

    driver.get('{}/ghazals'.format(author_page_hyperlink))
    elements = driver.find_elements_by_class_name('rt_bodyTitle')

    author_ghazal_hyperlinks = []
    for element in elements:
        for k in element.find_elements_by_tag_name('a'):
            if k.get_attribute('href')==None:
                pass
            else:
                author_ghazal_hyperlinks.append(k.get_attribute('href'))
    
    driver.close()
    
    return author_ghazal_hyperlinks

In [53]:
def get_ghazal_from_hyperlink(hyperlink):
    

    complete_ghazals = {
        'en': '',
        'hi': '',
        'ur': ''
    }
    for lang in complete_ghazals:
        driver = webdriver.Chrome()
        if lang=='en':
            
            driver.get(hyperlink)
        else:
            driver.get(hyperlink+'?lang='+lang)
            
    
        double_lines = driver.find_elements_by_class_name('c')

        check_word_text = False
        for double_line in double_lines:
            for line in double_line.find_elements_by_tag_name('p'):
                for word in line.find_elements_by_tag_name('span'):
                    if word.text==None or word.text=='':
                        check_word_text=False
                    else:
                        check_word_text=True
                    complete_ghazals[lang] +=word.text
                if check_word_text:
                    complete_ghazals[lang]+="\n"
            if check_word_text:
                complete_ghazals[lang]+="\n\n"
        driver.close()
    return complete_ghazals


In [34]:
author_profile_hyperlinks = get_all_author_profile_hyperlink("https://www.rekhta.org/poets/")

scrolling...
waiting
scrolling...
waiting
scrolling...
waiting
stopped scrolling - time


/home/reverie-pc/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  if sys.path[0] == '':
  0%|          | 0/1173 [00:00<?, ?it/s]/home/reverie-pc/anaconda3/envs/nlp/lib/python3.7/site-packages/selenium/webdriver/remote/webelement.py:359: UserWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  warnings.warn("find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead")
100%|██████████| 1173/1173 [00:24<00:00, 48.67it/s]


In [ ]:
big_data_dict = []
itr = 0
for author_profile_link in tqdm(author_profile_hyperlinks):
    author_name = author_profile_link.split('/')[-1]
    ghazal_hyperlinks = get_ghazal_hyperlinks_from_author_page(author_profile_link)
    
    for ghazal_link in ghazal_hyperlinks:
        ghazal_name = ghazal_link.split('/')[-1]
        
        ghazals = get_ghazal_from_hyperlink(ghazal_link)
        
        big_data_dict.extend([None]*len(ghazals.keys()))
        for lang in ghazals:
            
            big_data_dict[itr] = {
                "author_name":author_name,
                "ghazal_name": ghazal_name,
                "language": lang,
                "ghazal": ghazals[lang]}
            
            itr+=1

  0%|          | 0/1173 [00:00<?, ?it/s]/home/reverie-pc/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  """
/home/reverie-pc/anaconda3/envs/nlp/lib/python3.7/site-packages/selenium/webdriver/remote/webelement.py:359: UserWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  warnings.warn("find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead")
/home/reverie-pc/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
/home/reverie-pc/anaconda3/envs/nlp/lib/python3.7/site-packages/selenium/webdriver/remote/webelement.py:359: UserWarning: find_elements_by_tag_name is deprecated. Please use find

/home/reverie-pc/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
/home/reverie-pc/anaconda3/envs/nlp/lib/python3.7/site-packages/selenium/webdriver/remote/webelement.py:359: UserWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  warnings.warn("find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead")
/home/reverie-pc/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
/home/reverie-pc/anaconda3/envs/nlp/lib/python3.7/site-packages/selenium/webdriver/remote/webelement.py:359: UserWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead

/home/reverie-pc/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
/home/reverie-pc/anaconda3/envs/nlp/lib/python3.7/site-packages/selenium/webdriver/remote/webelement.py:359: UserWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  warnings.warn("find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead")
/home/reverie-pc/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
/home/reverie-pc/anaconda3/envs/nlp/lib/python3.7/site-packages/selenium/webdriver/remote/webelement.py:359: UserWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead

In [85]:
ghazal_hyperlinks

['https://www.rekhta.org/ghazals/guzre-jo-apne-yaaron-kii-sohbat-men-chaar-din-a-g-josh-ghazals',
 'https://www.rekhta.org/ghazals/zakhm-khaate-hain-aur-muskuraate-hain-ham-a-g-josh-ghazals',
 'https://www.rekhta.org/ghazals/chaandnii-raat-men-andheraa-thaa-a-g-josh-ghazals',
 'https://www.rekhta.org/ghazals/itnaa-ehsaan-to-ham-par-vo-khudaaraa-karte-a-g-josh-ghazals',
 'https://www.rekhta.org/ghazals/aah-bhii-harf-e-duaa-ho-jaise-a-g-josh-ghazals',
 'https://www.rekhta.org/ghazals/har-mulaaqaat-men-lagte-hain-vo-begaane-se-a-g-josh-ghazals',
 'https://www.rekhta.org/ghazals/jab-kabhii-zikr-yaar-kaa-aayaa-a-g-josh-ghazals',
 'https://www.rekhta.org/ghazals/ik-zaraa-tum-se-shanaasaaii-huii-a-g-josh-ghazals',
 'https://www.rekhta.org/ghazals/suunaa-suunaa-dil-kaa-mujhe-nagar-lagtaa-hai-a-g-josh-ghazals',
 'https://www.rekhta.org/ghazals/jiinaa-kab-tak-muhaal-hogaa-a-g-josh-ghazals',
 'https://www.rekhta.org/ghazals/na-sochnaa-ki-zamaane-se-dar-gae-ham-bhii-a-g-josh-ghazals',
 'https://ww

In [76]:
ghazals = get_ghazal_from_hyperlink(ghazal_hyperlinks[0])

/home/reverie-pc/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
/home/reverie-pc/anaconda3/envs/nlp/lib/python3.7/site-packages/selenium/webdriver/remote/webelement.py:359: UserWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  warnings.warn("find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead")
/home/reverie-pc/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
/home/reverie-pc/anaconda3/envs/nlp/lib/python3.7/site-packages/selenium/webdriver/remote/webelement.py:359: UserWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead

In [81]:
print(ghazals['hi'])

गुज़रे जो अपने यारों की सोहबत में चार दिन 
ऐसा लगा बसर हुए जन्नत में चार दिन 


उम्र-ए-ख़िज़र की उस को तमन्ना कभी न हो 
इंसान जी सके जो मोहब्बत में चार दिन 


जब तक जिए निभाएँगे हम उन से दोस्ती 
अपने रहे जो दोस्त मुसीबत में चार दिन 


ऐ जान-ए-आरज़ू वो क़यामत से कम न थे 
काटे तिरे बग़ैर जो ग़ुर्बत में चार दिन 


फिर उम्र भर कभी न सुकूँ पा सका ये दिल 
कटने थे जो भी कट गए राहत में चार दिन 


जो फ़क़्र में सुरूर है शाही में वो कहाँ 
हम भी रहे हैं नश्शा-ए-दौलत में चार दिन 


उस आग ने जला के ये दिल राख कर दिया 
उठते थे 'जोश' शोले जो वहशत में चार दिन 



